In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn.functional as F
import torch.cuda

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

df = pd.read_csv("/kaggle/input/imdb-reviews-sentiment4/imdb_reviews_sentiment3.csv")
#df.columns = ['label', 'text']
df.head()

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/imdb-reviews-sentiment4/imdb_reviews_sentiment3.csv
/kaggle/input/imdb-reviews-sentiment2/imdb_reviews_sentiment2.csv
/kaggle/input/cs39aa-airline-tweet-sentiment-classification/sample_submission.csv
/kaggle/input/cs39aa-airline-tweet-sentiment-classification/train.csv
/kaggle/input/cs39aa-airline-tweet-sentiment-classification/test.csv


In [2]:
df.rename(columns={'Review': 'text'}, inplace=True)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=50

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create a vocabulary for the y (dependent variable). Note which index corresponds to positive, negative, and to neutral. 

In [5]:
classes = df.sentiment.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'positive': 0, 'negative': 1, 'neutral': 2}
{0: 'positive', 1: 'negative', 2: 'neutral'}


Create a new column with these new labels. This will be the y that is used. 

Notice that all of the sample tweets below produce similar predictions, with each of them predicting roughly equal likelihood of pos/neg/neutral. This is because of the new layer that has been added to the model, and the fact that this new layer has its weight and bias parameters initialized to random values. We have to fine-tune this model (i.e. train this last layer) so that this model can start to make better predictions. 

In [6]:
df['label'] = df['sentiment'].apply(lambda x: class_tok2idx[x])
df.head()

,sentiment,text,label
0,positive,The villains in the predictable but entertaini...,0
1,positive,'Thunder Road' is a deftly presented tragicome...,0
2,positive,Dennis (Aml Ameen) watches his father (Everald...,0
3,positive,I usually give horror films allot of latitude ...,0
4,positive,Being a millenial I never got a chance to watc...,0


In [7]:
sequence_0 = "Do not watch this, it was full of terrible acting!"
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq0 is 14
{'input_ids': tensor([[ 101, 2091, 1136, 2824, 1142,  117, 1122, 1108, 1554, 1104, 6434, 3176,
          106,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[0.3790, 0.3015, 0.3195]], grad_fn=<SoftmaxBackward0>)

In [8]:
sequence_1 = "The whole movie was very good, but the ending was bad."
seq1_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq1_tokens['input_ids'].flatten())}")
F.softmax(model(**seq1_tokens).logits, dim=1)

number of tokens in seq1 is 15


tensor([[0.3723, 0.3446, 0.2830]], grad_fn=<SoftmaxBackward0>)

In [9]:
sequence_2 = "There is a reason why I only watch these movies,they're the best!"
seq2_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq2_tokens['input_ids'].flatten())}")
F.softmax(model(**seq2_tokens).logits, dim=1)

number of tokens in seq2 is 19


tensor([[0.3855, 0.3038, 0.3107]], grad_fn=<SoftmaxBackward0>)

In [10]:
sequence_3 = "Worst movie ever!"
seq3_tokens = tokenizer(sequence_3, return_tensors="pt")
print(f"number of tokens in seq3 is {len(seq3_tokens['input_ids'].flatten())}")
F.softmax(model(**seq3_tokens).logits, dim=1)

number of tokens in seq3 is 6


tensor([[0.3706, 0.3333, 0.2961]], grad_fn=<SoftmaxBackward0>)

In [11]:
ds_raw = Dataset.from_pandas(df[['label','text']])
ds_raw[0]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


{'label': 0,
 'text': 'The villains in the predictable but entertaining cliffhanger "Skyscraper" find themselves literally caught between a rock--Dwayne Johnson--and a hard place--the world\'s tallest building. "Central Intelligence" writer & director Rawson Marshall Thurber must have been thinking about the Bruce Willis thriller "Die Hard" (1988) and Irwin Allen\'s classic disaster epic "The Towering Inferno" (1974) when he cobbled together this derivative, white-knuckled hokum. While the CGI passes muster, Dwayne Johnson amounts to "Skyscraper\'s" the biggest special effect. The Rock\'s charismatic presence redeems this far-fetched, but formulaic caper with clichés galore. Nevertheless, Johnson challenges himself because his character has suffered the consequences for adversity. A former FBI Agent who almost bought the farm in a bomb blast when he cornered a suicide bomber, our hero lost his left leg below the knee and now sports a prosthetic leg. Thurber exploits this handicap for m

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

In [13]:
ds[0]

{'label': 0,
 'text': 'The villains in the predictable but entertaining cliffhanger "Skyscraper" find themselves literally caught between a rock--Dwayne Johnson--and a hard place--the world\'s tallest building. "Central Intelligence" writer & director Rawson Marshall Thurber must have been thinking about the Bruce Willis thriller "Die Hard" (1988) and Irwin Allen\'s classic disaster epic "The Towering Inferno" (1974) when he cobbled together this derivative, white-knuckled hokum. While the CGI passes muster, Dwayne Johnson amounts to "Skyscraper\'s" the biggest special effect. The Rock\'s charismatic presence redeems this far-fetched, but formulaic caper with clichés galore. Nevertheless, Johnson challenges himself because his character has suffered the consequences for adversity. A former FBI Agent who almost bought the farm in a bomb blast when he cornered a suicide bomber, our hero lost his left leg below the knee and now sports a prosthetic leg. Thurber exploits this handicap for m

In [14]:
ds = ds.shuffle(seed=42)
ds[0]

{'label': 0,
 'text': 'Obviously a lot of plot holes but if you can ignore them and enjoy the suspense it is pretty well acted and well shot5.8',
 'input_ids': [101,
  15580,
  170,
  1974,
  1104,
  4928,
  7996,
  1133,
  1191,
  1128,
  1169,
  8429,
  1172,
  1105,
  5548,
  1103,
  28117,
  21643,
  1122,
  1110,
  2785,
  1218,
  5376,
  1105,
  1218,
  2046,
  1571,
  119,
  129,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [15]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [16]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 7225
len(ds_eval) = 1275


In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [18]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="/kaggle/working",
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else: 
    device = "cpu"

Using GPU


In [20]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy
200,No log,0.433048,0.823529
400,No log,0.433453,0.820392
600,0.413900,0.443840,0.826667
800,0.413900,0.497223,0.816471
1000,0.244600,0.558225,0.821176
1200,0.244600,0.584274,0.822745
1400,0.244600,0.661775,0.814118
1600,0.135000,0.722390,0.818039
1800,0.135000,0.752950,0.813333
2000,0.070700,0.781798,0.814902


{'eval_loss': 0.7988459467887878,
 'eval_accuracy': 0.8117647058823529,
 'eval_runtime': 2.2175,
 'eval_samples_per_second': 574.962,
 'eval_steps_per_second': 18.038,
 'epoch': 10.0}

In [22]:
'''import matplotlib.pyplot as plt

# Possible values of min_samples_split are 10 to 70 (by 5)
hyp_param_vals = list(range(5,151,10))
#hyp_param_vals = [5] + list(range(10,201,10))
metrics = []
for hp in hyp_param_vals:
    metrics.append(fitRFModel(hp))

fig = plt.figure(figsize=(12, 6))
ax = fig.add_axes([0, 0, 1, 1]) #.1, 0.1, 0.8, 0.8]) # main axes
ax.plot(hyp_param_vals, [metric[3] for metric in metrics], '--ro') # validattion accuracy
ax.plot(hyp_param_vals, [metric[2] for metric in metrics], '--bo') # training accuracy
ax.legend(["Validation Accuracy", "Train Accuracy"], loc=4)
ax.set_xticks(hyp_param_vals)
ax.set(xlabel="n_estimators", ylabel="accuracy (higher is better)")
plt.show()'''

'import matplotlib.pyplot as plt\n\n# Possible values of min_samples_split are 10 to 70 (by 5)\nhyp_param_vals = list(range(5,151,10))\n#hyp_param_vals = [5] + list(range(10,201,10))\nmetrics = []\nfor hp in hyp_param_vals:\n    metrics.append(fitRFModel(hp))\n\nfig = plt.figure(figsize=(12, 6))\nax = fig.add_axes([0, 0, 1, 1]) #.1, 0.1, 0.8, 0.8]) # main axes\nax.plot(hyp_param_vals, [metric[3] for metric in metrics], \'--ro\') # validattion accuracy\nax.plot(hyp_param_vals, [metric[2] for metric in metrics], \'--bo\') # training accuracy\nax.legend(["Validation Accuracy", "Train Accuracy"], loc=4)\nax.set_xticks(hyp_param_vals)\nax.set(xlabel="n_estimators", ylabel="accuracy (higher is better)")\nplt.show()'

In [23]:
df_test = pd.read_csv("/kaggle/input/cs39aa-airline-tweet-sentiment-classification/test.csv")
df_test.head()

,id,text
0,0,@JetBlue okay awesome! Thank you!
1,1,@SouthwestAir wanting a ⭐️ for the ✈️ finding ...
2,2,"@united Funny thing happened, we arrived on ou..."
3,3,@united 2nd Late Flight flight today
4,4,@USAirways your service at PHL is abysmal. An ...


In [24]:
ds_test_raw = Dataset.from_pandas(df_test)
ds_test_raw[0]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


{'id': 0, 'text': '@JetBlue okay awesome! Thank you!'}

In [25]:
ds_test = ds_test_raw.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [26]:
preds = trainer.predict(test_dataset=ds_test)

In [27]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions) 
#ds_test['preds'] = pd.Series(test_preds.tolist())

In [28]:
df_test['preds'] = test_preds.tolist()
df_test['sentiment'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()

,id,text,preds,sentiment
0,0,@JetBlue okay awesome! Thank you!,0,positive
1,1,@SouthwestAir wanting a ⭐️ for the ✈️ finding ...,0,positive
2,2,"@united Funny thing happened, we arrived on ou...",1,negative
3,3,@united 2nd Late Flight flight today,0,positive
4,4,@USAirways your service at PHL is abysmal. An ...,1,negative


In [29]:
df_submission = df_test[['id','sentiment']]
df_submission.head()

,id,sentiment
0,0,positive
1,1,positive
2,2,negative
3,3,positive
4,4,negative


In [30]:
df_submission.to_csv('/kaggle/working/submission.csv', index=False)